# **Census API**

The purpose of this notebook is to familiarize ourselves with the Census API, found [here](https://www.census.gov/data/developers/data-sets/acs-1year.html).


## **Step 1: Set up API Key**<br>
To get access to the API, we need to request an API key [here](https://api.census.gov/data/key_signup.html). A few minutes after filling out this form, you will get an email with set up instructions.

## **Step 2: Set key value**<br>
Once you have the API key, set it down below. Make sure to keep this value private / delete it before uploading it anywhere.

In [1]:
# Keep this value SECRET
#API_SECRET_KEY = 'ca1c7f53595da174a093c91b1cf57786f2ba803e'
# API_SECRET_KEY = '43eeded75ef136a63b130ef012e7bd5c3bacf32a'
API_SECRET_KEY = '12bdbba3fa5fea2052b457fcc1fbc33dcd03607b'

## **Step 3: ACS API Handbook**

Refernce this documentation to gain an understanding of how to use the API.

[ACS API Handbook](https://www.census.gov/content/dam/Census/library/publications/2020/acs/acs_api_handbook_2020_ch02.pdf)

Below we're going to make a sample request to get some data from DP03. In the URL variable, you can change the year, and whether you want acs1, acs3, or acs5. Keep in mind, that different acs values may have different years of availability. If you click [here](https://www.census.gov/data/developers/data-sets/acs-3year.html) you can see more information on ACS-3 and some sample calls to different tables.

For this tutorial, we will focus on the *Data Profile (DP)* table.

```
params = {
    'get': 'group(DP03), NAME',
    'for': 'state:11',
    'key': API_SECRET_KEY
}
```
The code above makes a request for the data in DP03, state 11. State 11 represents the District of Columbia. group(DP03) gives us all the variables & values for DP03, a full list of variables is available [here](https://api.census.gov/data/2013/acs/acs3/profile/variables.html). 

In [3]:
import requests

# Set the API endpoint URL and parameters
url = 'https://api.census.gov/data/2019/acs/acs5/profile'
params = {
    'get': 'group(DP05),NAME', # group(table_name) ie group(DP05)
    'for': 'state:11', # State 11 is District of Columbia
    'key': API_SECRET_KEY
}

# Make the API request and get the response
response = requests.get(url, params=params)

# Check if the response was successful
if response.status_code == 200:
    # Extract the JSON data from the response
    data = response.json()
    # Print the first row of the data, which contains the column names
    print(data[0])
    # Print the second row of the data, which contains the data for the District of Columbia
    print(data[1])
else:
    # If the response was not successful, print the status code
    print('Error: ' + str(response.status_code))

['NAME', 'DP05_0001E', 'DP05_0001EA', 'DP05_0001M', 'DP05_0001MA', 'DP05_0001PE', 'DP05_0001PEA', 'DP05_0001PM', 'DP05_0001PMA', 'DP05_0002E', 'DP05_0002EA', 'DP05_0002M', 'DP05_0002MA', 'DP05_0002PE', 'DP05_0002PEA', 'DP05_0002PM', 'DP05_0002PMA', 'DP05_0003E', 'DP05_0003EA', 'DP05_0003M', 'DP05_0003MA', 'DP05_0003PE', 'DP05_0003PEA', 'DP05_0003PM', 'DP05_0003PMA', 'DP05_0004E', 'DP05_0004EA', 'DP05_0004M', 'DP05_0004MA', 'DP05_0004PE', 'DP05_0004PEA', 'DP05_0004PM', 'DP05_0004PMA', 'DP05_0005E', 'DP05_0005EA', 'DP05_0005M', 'DP05_0005MA', 'DP05_0005PE', 'DP05_0005PEA', 'DP05_0005PM', 'DP05_0005PMA', 'DP05_0006E', 'DP05_0006EA', 'DP05_0006M', 'DP05_0006MA', 'DP05_0006PE', 'DP05_0006PEA', 'DP05_0006PM', 'DP05_0006PMA', 'DP05_0007E', 'DP05_0007EA', 'DP05_0007M', 'DP05_0007MA', 'DP05_0007PE', 'DP05_0007PEA', 'DP05_0007PM', 'DP05_0007PMA', 'DP05_0008E', 'DP05_0008EA', 'DP05_0008M', 'DP05_0008MA', 'DP05_0008PE', 'DP05_0008PEA', 'DP05_0008PM', 'DP05_0008PMA', 'DP05_0009E', 'DP05_0009EA', 'D

## **Step 4: Convert JSON to DataFrame**

The object that we got from this GET request is a [requests.Response](https://www.w3schools.com/python/ref_requests_response.asp) object. We see that it's possible to convert this response object into JSON.

```
data = response.json()
```

Once we have our JSON object, we're now able to convert it into a Pandas DataFrame.

In [17]:
import pandas as pd
import requests

# Load the metadata files into Pandas DataFrames
metadata = pd.read_csv('ACSDP5Y2019.DP05-Column-Metadata.csv')
#variables = pd.read_csv('ACSDP5Y2019.DP05-Table-Notes.txt', delimiter='\t')

# Set the index of the metadata DataFrame to the variable code column
metadata.set_index('Column Name', inplace=True)

# Set the API endpoint URL and parameters
url = 'https://api.census.gov/data/2019/acs/acs5/profile'
params = {
    'get': 'group(DP05),NAME',
    'for': 'state:11',
    'key': '12bdbba3fa5fea2052b457fcc1fbc33dcd03607b'
}

# Make the API request and get the response
response = requests.get(url, params=params)

# Load the data into a Pandas DataFrame
df = pd.DataFrame(response.json())

# Set the first row as the column names
df.columns = df.iloc[0]
# Remove the first row
df = df[1:]

# Set the index to 'NAME' column
df.set_index('NAME', inplace=True)

# Replace -999999999 values with NaN
df.replace(-999999999, pd.NA, inplace=True)

# Create a dictionary of variable codes and their descriptions
variable_descriptions = metadata['Label'].to_dict()

# Rename the columns using the dictionary
df.rename(columns=variable_descriptions, inplace=True)

# Convert numerical columns to numeric type
df = df.apply(pd.to_numeric, errors='ignore')

# Display the cleaned dataframe
df.reset_index()


,NAME,Estimate!!SEX AND AGE!!Total population,Annotation of Estimate!!SEX AND AGE!!Total population,Margin of Error!!SEX AND AGE!!Total population,Annotation of Margin of Error!!SEX AND AGE!!Total population,Percent!!SEX AND AGE!!Total population,Annotation of Percent!!SEX AND AGE!!Total population,Percent Margin of Error!!SEX AND AGE!!Total population,Annotation of Percent Margin of Error!!SEX AND AGE!!Total population,Estimate!!SEX AND AGE!!Total population!!Male,...,"Estimate!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Female","Annotation of Estimate!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Female","Margin of Error!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Female","Annotation of Margin of Error!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Female","Percent!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Female","Annotation of Percent!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Female","Percent Margin of Error!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Female","Annotation of Percent Margin of Error!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Female",Geography,state
0,"(District of Columbia, District of Columbia)",692683,NaN,-555555555,*****,692683,NaN,-888888888,(X),328644,...,278053,NaN,1495,NaN,53.5,NaN,0.1,NaN,0400000US11,11


## **Step 5: Block Data**

In the previous section, we made a request for the totals within the "state" of the District of Columbia. Now we want to zoom in, and get data for each block. We will be switching tables to the 2020 Decennial Census (as DP03 doesn't appear to have block data).



In [33]:
# Set the API endpoint URL and parameters
url = 'https://api.census.gov/data/2020/dec/pl'
params = {
    'get': 'NAME',
    'for': 'block:*', # This gets all blocks
    'in' : 'state:11, county:*',
    'key': API_SECRET_KEY
}

# Make the API request and get the response
response = requests.get(url, params=params)

# Check if the response was successful
if response.status_code == 200:
    # Extract the JSON data from the response
    data = response.json()
    # Print the first row of the data, which contains the column names
    print(data[0])
    # Print the second row of the data, which contains the data for the District of Columbia
    print(data[1])
else:
    # If the response was not successful, print the status code
    print('Error: ' + str(response.status_code))

['NAME', 'state', 'county', 'tract', 'block']
['Block 2003, Block Group 2, Census Tract 53.03, District of Columbia, District of Columbia', '11', '001', '005303', '2003']


In [ ]:
df = pd.DataFrame(data)
df.head()

,0,1,2,3,4
0,NAME,state,county,tract,block
1,"Block 2006, Block Group 2, Census Tract 90, Di...",11,001,009000,2006
2,"Block 2007, Block Group 2, Census Tract 90, Di...",11,001,009000,2007
3,"Block 2008, Block Group 2, Census Tract 90, Di...",11,001,009000,2008
4,"Block 2009, Block Group 2, Census Tract 90, Di...",11,001,009000,2009
